# Generate training data for reference only. . .
# 生成训练数据，仅供参考。。。

In [ ]:
import os 
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
from tqdm import tqdm

In [ ]:
base_dir = pathlib.Path("/kaggle/input/hms-harmful-brain-activity-classification")
os.listdir(base_dir)

In [ ]:
train_eegs = "/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/"
train_spectrograms = "/kaggle/input/hms-harmful-brain-activity-classification/train_spectrograms/"
path_train = "/kaggle/input/hms-harmful-brain-activity-classification/train.csv"
output_size = 500

In [ ]:
ds_train = pd.read_csv(path_train)
ds_train.head(100)

In [ ]:
target_cols =['seizure_vote', 'lpd_vote','gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']

In [ ]:
temp_ds = ds_train.groupby(["eeg_id","spectrogram_id"])[target_cols].sum().reset_index()
temp_ds["all_vote"] = temp_ds[target_cols].sum(1)
for col in target_cols:
    temp_ds[col] = temp_ds[col]/temp_ds["all_vote"]
temp_ds = temp_ds.drop(columns = ["all_vote"])
temp_ds[temp_ds["eeg_id"] == 893864755]

In [ ]:
import math
import numpy
def resize_linear(image_matrix, new_height:int, new_width:int):
    """Perform a pure-numpy linear-resampled resize of an image."""
    output_image = numpy.zeros((new_height, new_width), dtype=image_matrix.dtype)
    original_height, original_width = image_matrix.shape
    inv_scale_factor_y = original_height/new_height
    inv_scale_factor_x = original_width/new_width

    # This is an ugly serial operation.
    for new_y in range(new_height):
        for new_x in range(new_width):
            # If you had a color image, you could repeat this with all channels here.
            # Find sub-pixels data:
            old_x = new_x * inv_scale_factor_x
            old_y = new_y * inv_scale_factor_y
            x_fraction = old_x - math.floor(old_x)
            y_fraction = old_y - math.floor(old_y)

            # Sample four neighboring pixels:
            left_upper = image_matrix[math.floor(old_y), math.floor(old_x)]
            right_upper = image_matrix[math.floor(old_y), min(image_matrix.shape[1] - 1, math.ceil(old_x))]
            left_lower = image_matrix[min(image_matrix.shape[0] - 1, math.ceil(old_y)), math.floor(old_x)]
            right_lower = image_matrix[min(image_matrix.shape[0] - 1, math.ceil(old_y)), min(image_matrix.shape[1] - 1, math.ceil(old_x))]

            # Interpolate horizontally:
            blend_top = (right_upper * x_fraction) + (left_upper * (1.0 - x_fraction))
            blend_bottom = (right_lower * x_fraction) + (left_lower * (1.0 - x_fraction))
            # Interpolate vertically:
            final_blend = (blend_top * y_fraction) + (blend_bottom * (1.0 - y_fraction))
            output_image[new_y, new_x] = final_blend
    return output_image

In [ ]:
list_train = []
for i in tqdm(range(temp_ds.shape[0])):
    eeg_id = temp_ds.loc[i,"eeg_id"]
    values_eeg = pd.read_parquet(train_eegs+str(eeg_id)+".parquet").values[:2000,:]
    spectrogram_id = temp_ds.loc[i,"spectrogram_id"]
    values_spc = pd.read_parquet(train_spectrograms+str(spectrogram_id)+".parquet").values[:4000,1:]
    values_spc=np.clip(values_spc,np.exp(-6),np.exp(10))#最大值为89209464.0
    values_spc= np.log(values_spc)#对数变换
    v1 = resize_linear(values_eeg,output_size,20)
    v2 = resize_linear(values_spc,output_size,100)
    #print(np.concatenate((v1, v2), axis=1).shape)
    list_train.append(np.concatenate((v1, v2), axis=1))

input_size = values.shape[0]
output_size = 500
bin_size = input_size // output_size
values.reshape((output_size, bin_size,values.shape[1])).mean(1)

In [ ]:
train_ds = np.array(list_train)

In [ ]:
import joblib

In [ ]:
joblib.dump(temp_ds[target_cols].values,"target.pkl")

In [ ]:
joblib.dump(np.nan_to_num(train_ds),"train_matrix.pkl")